In [9]:
# imports
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# importing sound classification models from torchaudio
import torchaudio

import matplotlib.pyplot as plt
import tqdm

In [10]:
# List the available devices:
print("Available devices:")
print(torch.cuda.device_count())
print("List of devices:")
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
#print(torch.cuda.device(1))
#print(torch.cuda.get_device_name(1))
device_ids = [0, 1]
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPUs

Available devices:
1
List of devices:
NVIDIA GeForce RTX 3050 Ti Laptop GPU


## Dataset

In [11]:
# Loading the dataset
## Structure of the dataset is as follows:
### dataset_folder -> Folder_name_of_one_class -> audio_file.wav
dataset_folder = '/kaggle/input/audioset/TrainSet'
animals_folder = dataset_folder + '/Animals'+'/animals_segmented'
music_folder = dataset_folder + '/Music'+'/instruments_segmented'
sot_folder = dataset_folder + '/SoT'+'/sound_of_things_segmented'

In [12]:
# Putting the data paths into dictionaries (key: class, value: list of file names)
data_paths = {
    'animals': [os.path.join(animals_folder, file) for file in os.listdir(animals_folder)],
    'music': [os.path.join(music_folder, file) for file in os.listdir(music_folder)],
    'sot': [os.path.join(sot_folder, file) for file in os.listdir(sot_folder)]
}

animals_dict = {
    "animals": 0,
    "music": 1,
    "sot": 2
}

# print len of each class
print(len(data_paths['animals']))
print(len(data_paths['music']))
print(len(data_paths['sot']))
print("Total number of files: ", len(data_paths['animals']) + len(data_paths['music']) + len(data_paths['sot']))

print(data_paths['animals'][:5])

# Tuple to tensor of numbers
def name_tuple_to_float_tensor(tuple):
    # Read the content of the tupe and use animals_dict to convert the class name to a number in a new tensor
    return torch.tensor([animals_dict[tuple[i]] for i in range(len(tuple))], dtype=torch.float32)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '/kaggle/input/audioset/TrainSet/Animals/animals_segmented'

In [ ]:
import torch
from torch.utils.data import Dataset
num_classes = 3
class AudioDataset(Dataset):
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.classes = list(data_paths.keys())
        self.files = sum([data_paths[cls] for cls in self.classes], [])
        self.transform = transform
        self.printPath = False

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        for cls in self.classes:
            if idx < len(self.data_paths[cls]):
                file = self.data_paths[cls][idx]
                classe = cls
                break
            idx -= len(self.data_paths[cls])
        else:
            raise IndexError('Index out of range')

        waveform, sample_rate = torchaudio.load(file)
        if self.transform:
            mfcc = self.transform(waveform)
        
        if True:
            # Taking a mean tensor of the mfcc 2 channel tensor
            mfcc = torch.mean(mfcc, dim=0).detach() 
    
        if self.printPath:
            return mfcc, sample_rate, classe, file
        else:
            return mfcc, sample_rate, classe

In [ ]:
from torchvision.transforms import Compose, RandomApply

transforms = torchaudio.transforms.MFCC(
    sample_rate= 48000,
    n_mfcc=20)

transforms_2 = Compose([
    RandomApply([torchaudio.transforms.PitchShift(sample_rate=48000, n_steps=2)], p=0.4),
    RandomApply([torchaudio.transforms.FrequencyMasking(freq_mask_param=15)], p=0.2),  # SpecAugment
    RandomApply([torchaudio.transforms.TimeMasking(time_mask_param=35)], p=0.2),  # SpecAugment
    torchaudio.transforms.MFCC(sample_rate=48000, n_mfcc=20),
    #torchaudio.transforms.SlidingWindowCmn(cmn_window=600, min_cmn_window=100, center=False, norm_vars=False), # https://dsp.stackexchange.com/questions/19564/cepstral-mean-normalization
])

transform_norm_and_MFFC = Compose([
    torchaudio.transforms.SlidingWindowCmn(cmn_window=600, min_cmn_window=100, center=False, norm_vars=False), # https://dsp.stackexchange.com/questions/19564/cepstral-mean-normalization
    torchaudio.transforms.MFCC(sample_rate=48000, n_mfcc=20),
])

In [ ]:
dataset1 = AudioDataset(data_paths, transform=transforms)
dataset2 = AudioDataset(data_paths, transform=transforms_2)
dataset3 = AudioDataset(data_paths, transform=transform_norm_and_MFFC)
mfcctensor, sample_rate, classe = dataset1.__getitem__(0)
print(mfcctensor.shape, sample_rate, classe)

print(mfcctensor)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

# Select one channel of the MFCC tensor
mfcc_channel_1 = mfcctensor.detach().numpy() # mfcctensor[0].detach().numpy()

# Plot the MFCC
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc_channel_1, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

In [ ]:
BATCH_SIZE = 32

train_size = int(0.8 * len(dataset1)) # + len(dataset2))) #int(0.8 * (len(dataset1) + len(dataset2) + len(dataset3)))
val_size = int(0.15 * len(dataset1)) #+ len(dataset2))) #int(0.1 * (len(dataset1) + len(dataset2) + len(dataset3)))
test_size = len(dataset1)  - train_size - val_size #  + len(dataset2) - train_size - val_size #len(dataset1) + len(dataset2) + len(dataset3) - train_size - val_size

dataset1.printPath = True
dataset2.printPath = True
dataset3.printPath = True

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset1, [train_size, val_size, test_size])  # + dataset2, [train_size, val_size, test_size]) # torch.utils.data.random_split(dataset1 + dataset2 + dataset3, [train_size, val_size, test_size])
print(len(train_dataset), len(val_dataset), len(test_dataset))
print(len(train_dataset) + len(val_dataset) + len(test_dataset))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
# get the first element of the dataloader
dataiter = iter(train_dataloader)
mfcctensor, sample_rate, classe, file = next(dataiter)
print(mfcctensor.shape, sample_rate, classe)

## Loading the model

In [ ]:
model_name = "csnn_simple_model"
# Load the model
model = torch.load("../Models/Trained on raw data/" + model_name + ".pth")

In [ ]:
# Test the model on 1 audio file
model.eval()
mfcctensor = mfcctensor.cuda()
output = model(mfcctensor)
print(output)
print(torch.argmax(output, dim=1))